<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/AI_inpaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#マスク作成

from PIL import Image, ImageOps

# ----------- 変数定義 -----------
path = "/content/10.png"
w, h = 1024, 768  # 出力サイズ
# --------------------------------

# 元画像読み込み
img = Image.open(path).convert("RGB")
w0, h0 = img.size
rw, rh = w / w0, h / h0

# サイズに応じてリサイズ＆パディング
if rw < rh:
  new_w = w
  new_h = int(h0 * rw)
  resized = img.resize((new_w, new_h), resample=Image.LANCZOS)
  pad_top = (h - new_h) // 2
  pad_bottom = h - new_h - pad_top
  pad = (0, pad_top, 0, pad_bottom)
else:
  new_h = h
  new_w = int(w0 * rh)
  resized = img.resize((new_w, new_h), resample=Image.LANCZOS)
  pad_left = (w - new_w) // 2
  pad_right = w - new_w - pad_left
  pad = (pad_left, 0, pad_right, 0)

# 余白追加済み画像の生成
image = ImageOps.expand(resized, border=pad, fill=(255, 255, 255))

# マスク画像（余白のみ白、他は黒）
mask = Image.new("L", (w, h), color=255)
mask.paste(0, box=(pad[0], pad[1], w - pad[2], h - pad[3]))

# 保存
!mkdir "/content/gen"
image.save("gen/padded_image.png")
mask.save("gen/mask_image.png")

In [ ]:
#!pip install transformers accelerate
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel
from diffusers.utils import load_image
import numpy as np
import torch

init_image = load_image(
    #"https://huggingface.co/datasets/diffusers/test-arrays/resolve/main/stable_diffusion_inpaint/boy.png"
  "/content/gen/padded_image.png"
)
init_image = init_image.resize((512, 512))

generator = torch.Generator(device="cpu").manual_seed(1)

mask_image = load_image(
    #"https://huggingface.co/datasets/diffusers/test-arrays/resolve/main/stable_diffusion_inpaint/boy_mask.png"
  "/content/gen/mask_image.png"
)
mask_image = mask_image.resize((512, 512))


def make_inpaint_condition(image, image_mask):
    image = np.array(image.convert("RGB")).astype(np.float32) / 255.0
    image_mask = np.array(image_mask.convert("L")).astype(np.float32) / 255.0

    assert image.shape[0:1] == image_mask.shape[0:1], "image and image_mask must have the same image size"
    image[image_mask > 0.5] = -1.0  # set as masked pixel
    image = np.expand_dims(image, 0).transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return image


control_image = make_inpaint_condition(init_image, mask_image)

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_inpaint", torch_dtype=torch.float16
)
"""pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
  "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16,
  safety_checker=None,
)

#pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
"""

prompt = "anime, colorful cloak, 1girl, floating, golden fox ears, golden long hair, elegant face, gentle smile, modest dress, natural fingers, starlit sky, magical glow, colorful light particles, soft rainbow haze, aurora sky, bluish shadows, faint magenta glow, delicate warm light"

# generate image
image = pipe(
    prompt,
    num_inference_steps=100,
    generator=generator,
    eta=1.0,
    image=init_image,
    mask_image=mask_image,
    control_image=control_image,

).images[0]

#pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))

image.save("output.png")